In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from ortools.linear_solver import pywraplp

# Step 1: Load Excel data
def load_player_data(file_path, sheet_name):
    # Load the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    return df

# Step 2: Extract relevant data and prepare it for optimization
def extract_player_data(df, player_name_col, attributes_cols):
    players = df[player_name_col].tolist()
    attributes = df[attributes_cols].values
    return players, attributes

# Step 3: Standardize the attributes
def standardize_attributes(attributes):
    scaler = StandardScaler()
    standardized_attributes = scaler.fit_transform(attributes)
    return standardized_attributes

# Step 4: Optimization function to select top 3 players
def select_best_midfielders(players, attributes, weights, num_players_to_select=3):
    # Create solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    n_players = len(players)

    # Decision variables: x[i] = 1 if player i is selected, 0 otherwise
    x = [solver.IntVar(0, 1, f'x_{i}') for i in range(n_players)]

    # Objective: Maximize weighted sum of standardized attributes for the selected players
    objective = solver.Objective()
    for i in range(n_players):
        score = sum(weights[j] * attributes[i][j] for j in range(len(weights)))
        objective.SetCoefficient(x[i], score)
    objective.SetMaximization()

    # Constraint: Select exactly num_players_to_select players
    solver.Add(sum(x) == num_players_to_select)

    # Solve the problem
    status = solver.Solve()

    # Get the best players
    if status == pywraplp.Solver.OPTIMAL:
        selected_players = []
        total_score = 0
        for i in range(n_players):
            if x[i].solution_value() == 1:
                selected_players.append((players[i], sum(weights[j] * attributes[i][j] for j in range(len(weights)))))
                total_score += sum(weights[j] * attributes[i][j] for j in range(len(weights)))
        return selected_players, total_score
    else:
        return None, None

# Example usage
if __name__ == "__main__":
    # File path and sheet name for your Excel file
    file_path = 'Z:/Yohan/Select the best midfielder/Stat.xlsx'
    sheet_name = 'basic data'

    # Load the player data from Excel
    df = load_player_data(file_path, sheet_name)
    
    df['Assists per game'] = df['All-time assists'] / df['All-time appearance']
    df['Premier league  Big Chances Created per game'] = df['Premier league  Big Chances Created'] / df['Premier league appearance']
    df['Premier league Accurate long balls per game'] = df['Premier league Accurate long balls'] / df['Premier league appearance']
    df['Premier league  Blocked shots per game'] = df['Premier league  Blocked shots'] / df['Premier league appearance']
    df['Premier league Duels won per game'] = df['Premier league Duels won'] / df['Premier league appearance']
    df['Premier league Aerial battles won per game'] = df['Premier league Aerial battles won'] / df['Premier league appearance']
    df['Total awards'] = (df['Time of Player of the Season'] + df['Time of player of Month'] + 
                          df['Time of Goal of the season'] + df['Time of Goal of the Month'] + 
                          df['Time of Premier League Champion'] + df['Time of Playmaker of year'])

    # Specify the column names for player names and the relevant attributes
    player_name_col = 'Player'  # Example: column with player names
    attributes_cols = ['All-time assists','Assists per game','Premier league  Big Chances Created per game', 'Premier league Accurate long balls per game',
                       'Average succesced dribbiles in domestic league (last 4 seasons)','Premier league  Free kicks scored','Premier league Passes per match'
                      ,'Premier league Cross accuracy %', 'Premier league  Tackle success %',
                       'Premier league  Blocked shots per game','Premier league Duels won per game','Premier league Aerial battles won per game',
                       'Total awards','Average rating (last 12 months)','All-time yellow cards','All-time red cards']  # Example: relevant attributes for an attacking midfielder

    # Extract players and their attribute values
    players, attributes = extract_player_data(df, player_name_col, attributes_cols)

    # Standardize the attributes
    standardized_attributes = standardize_attributes(attributes)

    # Specify weights for the attributes based on their importance
    weights = [10, 10, 9, 9, 8, 8, 6, 6, 4, 4, 4, 4, 4, 4, -2, -2]  # Adjust the weights as necessary

    # Select the top 6 midfielders using binary optimization
    num_players_to_select = 10
    best_midfielders, total_score = select_best_midfielders(players, standardized_attributes, weights, num_players_to_select)

    # Output the results
    if best_midfielders:
        # Sort players by their score in descending order
        best_midfielders.sort(key=lambda x: x[1], reverse=True)
        
        print(f"The top {num_players_to_select} players ranked by score are:")
        for rank, (player, score) in enumerate(best_midfielders, 1):
            print(f"{rank}. {player} with a score of {score:.2f}")
        
    else:
        print("No optimal solution found.")

        

C:\Users\Yohan\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


The top 10 players ranked by score are:
1. Kevin De Bruyne  with a score of 179.44
2. Bruno Fernandes  with a score of 105.28
3. James Maddison with a score of 81.52
4. Martin Ødegaard with a score of 39.02
5. Cole Palmer  with a score of 37.45
6. Bernardo Silva  with a score of 35.14
7. Lucas Paquetá  with a score of 34.67
8. Mohammed Kudus with a score of 34.66
9. Eberechi Eze with a score of 29.03
10. Emiliano Buendía with a score of 15.95
